## 케라스 자동차 연비 예측 모델

- Source : https://www.tensorflow.org/tutorials/keras/regression

### modules import

### 데이터 로드

### 데이터 확인

### 데이터 전처리


- 해당 데이터는 일부 데이터가 누락되어 있음

- 누락된 행 삭제

- "Origin" 범주형 데이터
  - 원-핫 인코딩(one-hot encoding) 진행

#### 검증 데이터셋 생성

#### 데이터 조사

#### 데이터의 특성과 레이블 분리

#### 데이터 정규화

### 모델 구성

### 샘플 데이터 확인

### 모델 학습

### 모델 학습 시각화

### EarlyStopping을 이용한 규제화

### 모델 평가

### 학습된 모델을 통한 예측